In [1]:
import bs4
import numpy as np
import pandas as pd

In [2]:
def coerce_percent(x):
    try:
        if float(x) <= 100:
            return float(x)
        else:
            return np.nan
    except ValueError:
        return np.nan

In [3]:
crime = pd.read_excel(
    "Table_8_Offenses_Known_to_Law_Enforcement_by_State_by_City_2018.xls", skiprows=3
)
crime = crime.drop(crime.tail(10).index)
crime["State"] = crime["State"].fillna(method="ffill")
crime = crime.dropna()
crime["violent_crime_rate"] = crime["Violent\ncrime"] / crime["Population"]
crime["state"] = crime["State"].apply(
    lambda x: "".join(char for char in x if char.isalpha()).lower().replace("city", "")
)
crime["city"] = crime["City"].apply(
    lambda x: "".join(char for char in x if char.isalpha()).lower().replace("city", "")
)
crime["city_state"] = crime[["city", "state"]].apply(lambda x: "".join(x), axis=1)
crime = crime[["city_state", "violent_crime_rate"]].set_index("city_state")[
    "violent_crime_rate"
]

In [4]:
demo = pd.read_csv(
    "ACSDP1Y2018.DP05_data_with_overlays_2020-08-23T064728.csv", skiprows=1
)
name = demo["Geographic Area Name"]
rel_col = [col for col in demo.columns if col.lower().startswith("percent estimate")]
demo = demo[rel_col]
demo = demo.applymap(coerce_percent)
demo = demo.dropna(axis=1, how="all")
demo["Geographic Area Name"] = name

In [5]:
econ = pd.read_csv(
    "ACSDP1Y2018.DP03_data_with_overlays_2020-08-23T070706.csv", skiprows=1
)
name = econ["Geographic Area Name"]
rel_col = [col for col in econ.columns if col.lower().startswith("percent estimate")]
econ = econ[rel_col]
econ = econ.applymap(coerce_percent)
econ = econ.dropna(axis=1, how="all")
econ["Geographic Area Name"] = name

In [6]:
census = pd.merge(econ, demo, on="Geographic Area Name")
census["state"] = census["Geographic Area Name"].apply(lambda x: x.split(",")[-1])
census["city"] = census["Geographic Area Name"].apply(lambda x: x.split(",")[0])
census["state"] = census["state"].apply(
    lambda x: "".join(char for char in x if char.isalpha()).lower().replace("city", "")
)
census["city"] = census["city"].apply(
    lambda x: "".join(char for char in x if char.isalpha()).lower().replace("city", "")
)
census["city_state"] = census[["city", "state"]].apply(lambda x: "".join(x), axis=1)
census = census.drop(["Geographic Area Name", "state", "city"], axis=1)
census = census.set_index("city_state")

In [7]:
file = open(
    "List of mayors of the 50 largest cities in the United States - Wikipedia.htm",
    "r",
    encoding="utf-8",
)
page = file.read()
file.close()
soup = bs4.BeautifulSoup(page)
mayor = []
for tag in soup.find_all("table")[1].find_all("tbody")[0].find_all("tr"):
    party = tag.find_all("td")[1].text
    city = tag.find_all("td")[2].text
    state = tag.find_all("td")[3].text
    mayor.append([party, city, state])
mayor = pd.DataFrame(mayor, columns="party city state".split())
mayor["city"] = mayor["city"].apply(
    lambda x: "".join(char for char in x if char.isalpha()).lower().replace("city", "")
)
mayor["state"] = mayor["state"].apply(
    lambda x: "".join(char for char in x if char.isalpha()).lower().replace("city", "")
)
mayor["city_state"] = mayor[["city", "state"]].apply(lambda x: "".join(x), axis=1)
mayor = mayor[["city_state", "party"]].set_index("city_state")["party"]

In [8]:
data = census.copy()
data["party"] = mayor
data["violent_crime_rate"] = crime
data = data.dropna(subset=["violent_crime_rate"])
data = pd.get_dummies(data, dummy_na=True)
data.loc[
    data["party_nan"] == 1,
    ["party_Democratic", "party_Independent", "party_Republican"],
] = np.nan
del data["party_nan"]
data = data.fillna(data.mean())
data.to_csv("data.csv")